**# 层和块**

**1. 回顾多层感知机**

* **nn.Sequential**定义了一种特殊的Module

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.0035,  0.0592,  0.0117,  0.1352,  0.0147,  0.0251,  0.0829,  0.0743,
         -0.0694, -0.0011],
        [ 0.0808, -0.0017, -0.0684,  0.1099, -0.0208,  0.0246, -0.0055,  0.1415,
          0.0284, -0.0937]], grad_fn=<AddmmBackward0>)

**2. 自定义块**

In [2]:
class MLP(nn.Module):
    # 用模型参数声明层。这里声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

* 实例化多层感知机的层，然后在每次调用前向传播函数时调用这些层。

In [3]:
net = MLP()
net(X)

tensor([[ 0.1748,  0.0943, -0.0814, -0.0205,  0.1919, -0.2706, -0.0943, -0.1047,
         -0.0311, -0.0616],
        [ 0.1664,  0.0964, -0.1545,  0.0232,  0.1502, -0.3235, -0.0822,  0.0050,
         -0.0689, -0.0586]], grad_fn=<AddmmBackward0>)

* 块的一个主要优点是它的多功能性。 可以子类化块以创建层（如全连接层的类）、 整个模型（如上面的MLP类）或具有中等复杂度的各种组件。

**3. 顺序块**

*  为了构建自己的简化的MySequential， 只需要定义两个关键函数：
  * 一种将块逐个追加到列表中的函数。
  * 一种前向传播函数，用于将输入按追加块的顺序传递给块组成的“链条”。

* 下面的MySequential类提供了与默认Sequential类相同的功能。

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        # 变量_modules中, module的类型是OrderedDict
        for block in args:
            self._modules[block] = block

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

* `__init__`函数将每个模块逐个添加到有序字典_modules中。 
* 简而言之，_modules的主要优点是：在模块的参数初始化过程中， 系统知道在_modules字典中查找要初始化参数的子块。
* 当MySequential的前向传播函数被调用时， 每个添加的块都按照它们被添加的顺序执行。 

In [6]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0482,  0.0072, -0.1293,  0.0820, -0.1496, -0.1181, -0.1499, -0.2098,
          0.0285, -0.1114],
        [-0.0939,  0.0924, -0.1982,  0.1054, -0.2458, -0.0868, -0.1588, -0.1632,
         -0.0462, -0.1426]], grad_fn=<AddmmBackward0>)

**4. 在前向传播函数中执行代码**

* Sequential类使模型构造变得简单， 允许我们组合新的架构，而不必定义自己的类。 然而，并不是所有的架构都是简单的顺序架构。 当需要更强的灵活性时，我们需要定义自己的块。 例如，我们可能希望在前向传播函数中执行Python的控制流。 此外，我们可能希望执行任意的数学运算，而不是简单地依赖预定义的神经网络层。

* 到目前为止，网络中的所有操作都对网络的激活值及网络的参数起作用。然而，有时可能希望合并既不是上一层的结果也不是可更新参数的项，称之为*常数参数*（constant parameter）。
例如，需要一个计算函数$f(\mathbf{x},\mathbf{w}) = c \cdot \mathbf{w}^\top \mathbf{x}$的层，其中$\mathbf{x}$是输入，$\mathbf{w}$是参数，$c$是某个在优化过程中没有更新的指定常量。因此实现了一个`FixedHiddenMLP`类，如下所示：

In [11]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

* 在这个`FixedHiddenMLP`模型中，实现了一个隐藏层，其权重（`self.rand_weight`）在实例化时被随机初始化，之后为常量。这个权重不是一个模型参数，因此它永远不会被反向传播更新。
然后，神经网络将这个固定层的输出通过一个全连接层。

* 注意，在返回输出之前，模型做了一些不寻常的事情：它运行了一个while循环，在$L_1$范数大于$1$的条件下，将输出向量除以$2$，直到它满足条件为止。最后，模型返回了`X`中所有项的和。此操作可能不会常用于在任何实际任务中，只是展示如何将任意代码集成到神经网络计算的流程中。


**5. 混合搭配各种组合块的方法**

In [12]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

In [13]:
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.2484, grad_fn=<SumBackward0>)